In [1]:
import os
import h5py
import numpy as np
import torch
from tensorflow.keras import models, layers, preprocessing, datasets
from torch.utils.data import Dataset, DataLoader
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.transforms import Compose, LoadImage, EnsureChannelFirst, ScaleIntensity, ToTensor
from sklearn.model_selection import train_test_split
import cv2
from torch.optim import Adam

In [2]:
def imageGenerator():
    input_folder = r'C:\Users\marto\Desktop\Szakdolgozat\pogram\brainTumor\data'
    output_folder = r'C:\Users\marto\Desktop\Szakdolgozat\pogram\brainTumor\converted_data'
    datas = []
    img_size = (256,256)
    for file_name in sorted(os.listdir(input_folder)):
        if file_name.endswith('.mat'):
            file_path = os.path.join(input_folder,file_name)
            
            with h5py.File(file_path, 'r') as f:
                image = np.array(f['cjdata/image']).T
                label = int(np.array(f['cjdata/label'])[0,0])
    
            image = image.astype(np.float64)
            im_norm = 255*(image-image.min())/(image.max()-image.min())
            im_uint8 = im_norm.astype(np.uint8)

            file_name_base = os.path.splitext(file_name)[0]
            output_file_path = os.path.join(output_folder,file_name_base + '.jpg')
    
            im_pil = Image.fromarray(im_uint8)
            im_pil = im_pil.resize(img_size)
            datas.append((np.array(im_pil,dtype=np.float32),label))
            
            im_pil.save(output_file_path)
    #return datas

In [3]:
#imageGeneratior()

In [4]:
def imageProcesser():
    input_folder = r'C:\Users\marto\Desktop\Szakdolgozat\pogram\brainTumor\data'
    datas = []
    img_size = (256,256)
    for file_name in sorted(os.listdir(input_folder)):
        if file_name.endswith('.mat'):
            file_path = os.path.join(input_folder,file_name)
            
            with h5py.File(file_path, 'r') as f:
                label = int(np.array(f['cjdata/label'])[0,0])-1
                image = np.array(f['cjdata/image']).T
                tumor_border = np.array(f['cjdata/tumorBorder'])
                tumor_mask = np.array(f['cjdata/tumorMask']).T
                
            eps = 1e-8
            
            image = image.astype(np.float64)
            image = 255*(image-image.min())/(image.max()-image.min() + eps)
            image = cv2.resize(image,img_size)
            image_norm = image.astype(np.float32)/255.0
            image = np.expand_dims(image_norm,axis=0)

            tumor_mask = tumor_mask.astype(np.float64)
            tumorMask = 255*(tumor_mask-tumor_mask.min())/(tumor_mask.max()-tumor_mask.min() + eps)
            tumorMask = cv2.resize(tumorMask,img_size)
            tumor_mask_norm = tumorMask.astype(np.float32)/255.0
            mask = np.expand_dims(tumor_mask_norm,axis=0)

            
            datas.append((image,mask))
    return datas

In [5]:
datas = imageProcesser()
X = datas[0]
y = datas[1]

In [6]:
datageneration = preprocessing.image.ImageDataGenerator(
    rotation_range=20,       # forgatás (fokban)
    width_shift_range=0.1,   # vízszintes eltolás
    height_shift_range=0.1,  # függőleges eltolás
    zoom_range=0.1,          # nagyítás/kicsinyítés
    horizontal_flip=True,    # vízszintes tükrözés
    fill_mode='nearest'  
)

In [ ]:
#https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/brats_segmentation_3d.ipynb

In [7]:
train_size = int(0.8*len(datas))
val_size = len(datas) - train_size
train_datas, val_datas = torch.utils.data.random_split(datas,[train_size,val_size])

train = DataLoader(train_datas,batch_size=8,shuffle=True)
val = DataLoader(val_datas,batch_size=8,shuffle=True)

In [8]:
model = UNet(
    spatial_dims=2,
    in_channels=1,       
    out_channels=1,      
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to("cpu")

In [9]:
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
dice_metric = DiceMetric(include_background=False, reduction="mean")

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [11]:
loss_function = DiceLoss(sigmoid=True)
optimizer = Adam(model.parameters(), lr=1e-4)

for epoch in range(30):
    model.train()
    epoch_loss = 0
    for i, (images, masks) in enumerate(train):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(train)}")

Epoch 1, Loss: 0.9561275457326286
Epoch 2, Loss: 0.9419602126562634
Epoch 3, Loss: 0.9308613626110437
Epoch 4, Loss: 0.9210789836579115
Epoch 5, Loss: 0.9121567362294524
Epoch 6, Loss: 0.9028319930020683
Epoch 7, Loss: 0.8931077377027331
Epoch 8, Loss: 0.8837340594502925
Epoch 9, Loss: 0.8732433441407518
Epoch 10, Loss: 0.8611405860717599
Epoch 11, Loss: 0.8459513507759144
Epoch 12, Loss: 0.8287786303591651
Epoch 13, Loss: 0.8100720179974062
Epoch 14, Loss: 0.7882116347648421
Epoch 15, Loss: 0.7523500414547004
Epoch 16, Loss: 0.7194694080647894
Epoch 17, Loss: 0.6882811058616017
Epoch 18, Loss: 0.6572260359599458
Epoch 19, Loss: 0.6251351575121429
Epoch 20, Loss: 0.5951693163826722
Epoch 21, Loss: 0.5697724899756403
Epoch 22, Loss: 0.5381044614392694
Epoch 23, Loss: 0.5072611624720819
Epoch 24, Loss: 0.4784121952150078
Epoch 25, Loss: 0.4503126313321365
Epoch 26, Loss: 0.42461157854682846
Epoch 27, Loss: 0.4016454966817694
Epoch 28, Loss: 0.3783820208878005
Epoch 29, Loss: 0.3551964073